**Función:** Entrenamiento de la CNN línea base (con dataset 10k desbalanceado).

**Requiere:**
- ../../datasets/X_train_DATASET10K224_x_float.pkl
- ../../datasets/X_test_DATASET10K224_x_float.pkl
- ../../datasets/y_train_DATASET10K224.pkl
- ../../datasets/y_test_DATASET10K224.pkl

**Crea:** Fichero con el modelo entrenado (epoch con menor val_loss).
- modelos/tf_model_\<run_id\>

In [12]:
ejecutadoEnGoogleDrive = False # True si se ejecuta en Google Drive

In [ ]:
if ejecutadoEnGoogleDrive:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = '/content/drive/clasificacion_imagenes/'
    ruta_datasets = '/content/drive/clasificacion_imagenes/datasets/'
else:
    PATH = '../../1_Setup_MLOps'
    ruta_datasets = '../../datasets/'
%cd {PATH}

In [ ]:
# !pip install mlflow
import mlflow
#from google.colab import drive
import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D
from tensorflow import keras
from tensorflow.random import set_seed
import time

In [ ]:
seed(42) # So long and thanks for all the fish
set_seed(42)

In [ ]:
# el experimento 1 tiene el id "0" (string)
experiment_id = "0"

In [ ]:
# Definir quién está ejecutando (para no machacar el MLflow de los compañeros)
# Descomentar la linea que aplique
# yo = 'Pedro_T'
# yo = 'Toni_V'
# yo = 'Carlos_H'
# yo = 'Carlos_C'
print('Notebook ejecutado por {}'.format(yo))

Notebook ejecutado por Pedro_T


In [ ]:
mlflow.set_experiment(experiment_id = experiment_id) 

<Experiment: artifact_location='mlruns', experiment_id='0', lifecycle_stage='active', name='Entrenamiento de CNN', tags={'Dataset': 'DATASET 10K 224',
 'Experimento': '1',
 'Modelo': 'CNN basica (baseline)',
 'mlflow.note.content': 'En este experimento entrenamos una CNN con el '
                        'DATASET10K224 de cara a establecer la línea base del '
                        'trabajo final del Máster en Data Science (MDS) en The '
                        'Valley Business School.'}>

In [ ]:
# reanudad run 2eb65387888c4bde8a458c9fc544bbb2
run_id = # por ej. "69bfdf296f384a75ada2ae55f9dc7672" (ver en la carpeta mlruns/0)
run = mlflow.start_run(run_id=run_id)

In [ ]:
# crear run
solo si aplica! (comentar esta linea > ejecutar > descomentar)
run = mlflow.start_run(
  experiment_id=experiment_id,
  tags={"Autor": yo}
)
print('Run creado')

Run creado


In [ ]:
# ver id de este run
print('Vamos empezar el run {} en el experimento {}. Este run está {}.'.format(run.info.run_id, experiment_id, run.info.status))

Vamos empezar el run 2eb65387888c4bde8a458c9fc544bbb2 en el experimento 0. Este run está RUNNING.


In [ ]:
# activar auto logging
mlflow.tensorflow.autolog()

Formato esperado:
- X_train: (N, 224, 224, 3) float32 (entre 0. y 1.)
- y_train: (N, 5) float32 (0. o 1.)
- X_test: (P, 224, 224, 3) float32 (entre 0. y 1.)
- y_test: (P, 5) float32 (0. o 1.)

In [ ]:
X_train = pickle.load(open(ruta_datasets + 'X_train_DATASET10K224_x_float.pkl','rb'))
X_test = pickle.load(open(ruta_datasets + 'X_test_DATASET10K224_x_float.pkl','rb'))
y_train = pickle.load(open(ruta_datasets + 'y_train_DATASET10K224.pkl','rb'))
y_test = pickle.load(open(ruta_datasets + 'y_test_DATASET10K224.pkl','rb'))

In [ ]:
categories = ['food', 'inside', 'outside', 'drink', 'menu']

In [ ]:
print("X_train: {} {}".format(X_train.shape, X_train.dtype))
print("y_train: {} {}".format(y_train.shape, y_train.dtype))
print("X_test: {} {}".format(X_test.shape, X_test.dtype))
print("y_test: {} {}".format(y_test.shape, y_test.dtype))

X_train: (6650, 224, 224, 3) float32
y_train: (6650, 5) float32
X_test: (1662, 224, 224, 3) float32
y_test: (1662, 5) float32


In [ ]:
# Dimensiones de la imagen de entrada
img_rows, img_cols, img_ch = 224, 224, 3
num_labels = 5

In [ ]:
# test de los datos de entrada
if   (X_train.shape[1] != img_rows) \
  or (X_train.shape[2] != img_cols)\
  or (X_train.shape[3] != img_ch)\
  or (y_train.shape[1] != num_labels)\
  or (X_train.dtype != 'float32')\
  or (X_test.shape[1] != img_rows) \
  or (X_test.shape[2] != img_cols)\
  or (X_test.shape[3] != img_ch)\
  or (y_test.shape[1] != num_labels)\
  or (X_test.dtype != 'float32')\
  or (y_train.dtype != 'float32')\
  or (y_test.dtype != 'float32')\
  or (X_train.shape[0] != y_train.shape[0])\
  or (X_test.shape[0] != y_test.shape[0]\
  or type(X_train) != np.ndarray\
  or type(X_test) != np.ndarray\
  or type(y_train) != np.ndarray\
  or type(y_test) != np.ndarray):
  raise Exception("El dataset no tiene las dimensiones esperadas!")
else:
  print('El dataset tiene las dimensiones esperadas')

El dataset tiene las dimensiones esperadas


In [ ]:
# función de Javier de la Rosa
def plot_hist(history, title="Model Accuracy"):
    """Imprime una gráfica mostrando la accuracy por epoch obtenida en un entrenamiento"""
    f, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
    ax1.plot(history.history['accuracy'])
    ax1.set_title("Model Accuracy")
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    
    ax2.plot(history.history['loss'])
    ax2.set_title("Model Loss")
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')

In [ ]:
IMG_SIZE=224
epochs = 40
batch_size=64
validation_split = 0.2

In [ ]:
model2 = Sequential()
model2.add(Conv2D(10, (12, 12), padding='same', input_shape=(IMG_SIZE,IMG_SIZE,3)))
model2.add(Activation('relu'))
model2.add(Conv2D(IMG_SIZE, (3, 3)))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(4, 4)))
model2.add(Dropout(0.25))

model2.add(Conv2D(IMG_SIZE, (3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(Conv2D(IMG_SIZE, (3, 3)))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(512))
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(5))
model2.add(Activation('softmax'))
opt = keras.optimizers.Nadam(lr=0.0001, schedule_decay=1e-6)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/nadam.py:78: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


In [ ]:
# las métricas han sido quitadas de Keras en 2.0
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model2.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc', f1_m, precision_m, recall_m])

In [ ]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 224, 224, 10)      4330      
                                                                 
 activation_6 (Activation)   (None, 224, 224, 10)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 222, 222, 224)     20384     
                                                                 
 activation_7 (Activation)   (None, 222, 222, 224)     0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 55, 55, 224)      0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 55, 55, 224)       0         
                                                      

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                         patience=8,
                                         restore_best_weights=True)

In [ ]:
start_time = time.time()
history = model2.fit(X_train, y_train,
                    epochs = epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test),
                    callbacks=[callback],
                    verbose=1)
end_time = time.time()

Epoch 1/40
104/104 [==============================] - 1537s 15s/step - loss: 0.9551 - acc: 0.6486 - f1_m: 0.6004 - precision_m: 0.7239 - recall_m: 0.5255 - val_loss: 0.8668 - val_acc: 0.7178 - val_f1_m: 0.5964 - val_precision_m: 0.7290 - val_recall_m: 0.5204
Epoch 2/40
104/104 [==============================] - 1571s 15s/step - loss: 0.8120 - acc: 0.7180 - f1_m: 0.6918 - precision_m: 0.7783 - recall_m: 0.6260 - val_loss: 0.7860 - val_acc: 0.7335 - val_f1_m: 0.6805 - val_precision_m: 0.7633 - val_recall_m: 0.6184
Epoch 3/40
104/104 [==============================] - 1405s 14s/step - loss: 0.7411 - acc: 0.7415 - f1_m: 0.7268 - precision_m: 0.7973 - recall_m: 0.6695 - val_loss: 0.8194 - val_acc: 0.7527 - val_f1_m: 0.6595 - val_precision_m: 0.8020 - val_recall_m: 0.5613
Epoch 4/40
104/104 [==============================] - 1444s 14s/step - loss: 0.6773 - acc: 0.7627 - f1_m: 0.7496 - precision_m: 0.8132 - recall_m: 0.6968 - val_loss: 0.6436 - val_acc: 0.7780 - val_f1_m: 0.7327 - val_precisi

INFO:tensorflow:Assets written to: /tmp/tmpvc94y6aj/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpvc94y6aj/model/data/model/assets
2022/09/16 16:07:35 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpvc94y6aj/model, flavor: keras), fall back to return ['tensorflow==2.9.1', 'keras==2.9.0']. Set logging level to DEBUG to see the full traceback.


In [ ]:
print(history.params)

{'verbose': 1, 'epochs': 40, 'steps': 104}


In [ ]:
history.history

{'loss': [0.955109715461731,
  0.8119623064994812,
  0.7411442399024963,
  0.6772661209106445,
  0.5923296809196472,
  0.5379163026809692,
  0.4737958610057831,
  0.38729074597358704,
  0.32863783836364746,
  0.26847633719444275,
  0.19956795871257782,
  0.25088316202163696,
  0.24121803045272827,
  0.0945020392537117,
  0.06864945590496063,
  0.05016828700900078,
  0.05161503702402115,
  0.041540127247571945,
  0.02806752920150757],
 'acc': [0.6485714316368103,
  0.7180451154708862,
  0.7415037751197815,
  0.7627067565917969,
  0.7837594151496887,
  0.8078195452690125,
  0.8278195261955261,
  0.8598496317863464,
  0.8837593793869019,
  0.9048120379447937,
  0.9326315522193909,
  0.9180451035499573,
  0.919248104095459,
  0.9714285731315613,
  0.9800000190734863,
  0.9860150218009949,
  0.9845112562179565,
  0.9885714054107666,
  0.9915789365768433],
 'f1_m': [0.6004496812820435,
  0.6918430328369141,
  0.7267724275588989,
  0.7496023774147034,
  0.7765404582023621,
  0.799664855003356

In [ ]:
print(history.history.keys())

dict_keys(['loss', 'acc', 'f1_m', 'precision_m', 'recall_m', 'val_loss', 'val_acc', 'val_f1_m', 'val_precision_m', 'val_recall_m'])


In [ ]:
% ir de PATH a Experimento_1/modelos
%cd ../4_Modelacion/Experimento_1/modelos/

In [ ]:
model2.save(PATH + 'tf_model_{}'.format(run.info.run_id))

INFO:tensorflow:Assets written to: Experimento_1/tf_model_2eb65387888c4bde8a458c9fc544bbb2/assets


INFO:tensorflow:Assets written to: Experimento_1/tf_model_2eb65387888c4bde8a458c9fc544bbb2/assets


In [ ]:
print('Guardado: Experimento_1/modelos/tf_model_{}'.format(run.info.run_id))

Guardado: Experimento_1/tf_model_2eb65387888c4bde8a458c9fc544bbb2


In [ ]:
mlflow.end_run()